In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
from bs4 import BeautifulSoup
import requests
import re

In [62]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [78]:
tokens = tokenizer.encode("this is good, but could be better", return_tensors='pt')

In [79]:
result = model(tokens)
result.logits

tensor([[-2.2128, -0.3438,  2.0223,  1.5271, -0.9159]],
       grad_fn=<AddmmBackward0>)

In [80]:
int(torch.argmax(result.logits)+1)


3

In [109]:
#collect reviews 
r = requests.get('https://de.trustpilot.com/review/www.saphe.com')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_body-l.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]
 

In [115]:
reviews[9]

'Ich bin insgesamt sehr zufrieden mit meinem Kauf des Saphe Gefahrenwarners. Der gesamte Prozess – vom Kauf bis zur Nutzung – verlief reibungslos. Die Installation war kinderleicht, und die Verbindung mit der App klappte auf Anhieb. Das Gerät ist kompakt, unauffällig und passt problemlos ins Fahrzeug.Besonders beeindruckt bin ich von der schnellen und zuverlässigen Warnung vor Gefahrenstellen und Blitzern. Es gibt ein gutes Gefühl, sicherer und entspannter unterwegs zu sein. Die Echtzeit-Updates sind ein großer Pluspunkt, da sie helfen, stets aktuelle Informationen zu erhalten.Ein kleiner Wermutstropfen war, dass ich bei zwei Blitzern erst gewarnt wurde, als ich bereits daran vorbeigefahren war – obwohl alle Einstellungen korrekt waren. Das ist jedoch eine Kleinigkeit und mindert meine Gesamtzufriedenheit nicht wesentlich.Alles in allem bietet Saphe eine hervorragende Lösung für sicheres Fahren. Ich kann das Produkt jedem empfehlen, der Wert auf zusätzliche Sicherheit und aktuelle Warn

In [116]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [120]:
df['review'].iloc[2]

'Ich habe es getestet für den pkw und bin positiv begeistert es sagt alles an und warnt rechtzeitig . Vorschlag sowas speziel für lkw wäre eine gute Sache die ich dann auch sofort testen würde '

In [121]:
def sentiment_pred(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)+1)


In [122]:
sentiment_pred(df['review'].iloc[2])

5

In [123]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_pred(x[:512]))

In [124]:
df

,review,sentiment
0,"4,3",4
1,Insgesamt 11.602,1
2,Ich habe es getestet für den pkw und bin posit...,5
3,Gut ist der Display der die Geschwindigkeit un...,4
4,"Das Gerät an sich ist recht gut, Akku hält lan...",4
5,Das Gerät hat bisher immer zuverlässig alle Bl...,5
6,Der Saphe ist gut zu bedienen übersichtlich un...,4
7,Was einfach nervt an den Geräten ist das selbe...,1
8,Soweit ganz zufrieden. Will sich immer gleich ...,4
9,Ich bin insgesamt sehr zufrieden mit meinem Ka...,5
